# 네이버 매장정보 크롤링 - 전처리 필요 파일


In [1]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  
import chromedriver_autoinstaller


# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
from tqdm import tqdm  

데이터 가져오기

In [29]:
# 데이터 불러오기
original_res_df = pd.read_excel('data/res_info_data(2).xlsx')
original_res_df = original_res_df.copy()
original_res_df = original_res_df.loc[original_res_df['업태구분'] == '전처리필요'].reset_index(drop=True)
original_res_df.drop('담당', axis=1, inplace=True)
original_res_df.head(10)

KeyboardInterrupt: 

In [31]:
re_res_df = original_res_df.copy()
start_num = 0 # 할 차례, 여기를 계속 수정해주세요. 
num = 100 # 몇개씩 크롤링할지(기본은 100개를 추천합니다. -> 16분)
end_num = start_num + num
res_df = re_res_df[start_num:end_num] # 맨 마지막 크롤링할떄는 end_num을 지우세요. [start_num:] <-이렇게 
res_df = res_df.reset_index(drop=True)
res_df.head(3)

,식당이름,업태구분,주소,메뉴,가격,방문자리뷰,블로그리뷰,검색어
0,언주로134길 29 폭풍갈비,전처리필요,전처리필요,메뉴없음,가격없음,0,0,언주로134길 29 폭풍갈비
1,"도산대로13길 14, 비즈니스아츠 디자인",전처리필요,전처리필요,메뉴없음,가격없음,0,0,"도산대로13길 14, 비즈니스아츠 디자인"
2,학동로 221 나주곰탕,전처리필요,전처리필요,메뉴없음,가격없음,0,0,학동로 221 나주곰탕


driver 세팅

In [26]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정 
driver = webdriver.Chrome(ChromeDriverManager().install())

# chromedriver_autoinstaller.install()
# driver = webdriver.Chrome()

크롤링 시작

In [27]:
# 가져올 정보들에 대한 리스트 지정
# 가게명, 업태명, 주소, 주력메뉴(맨첫번째메뉴) + 가격까지, 방문자리뷰수,블로그리뷰수

restaurant_name_list = []
category_name_list = []
address_list = []
menu_list = []
price_list = []
visitor_review_list =[]
blog_review_list = []

wait_time = 4

# 크롤링 시작합니다.
total_iterations = len(res_df) # 크롤링 진행 과정을 파악하기 위해 tqdm 사용 
for i in tqdm(range(total_iterations), desc='진행 중', position=0, leave=True):


    # 검색어를 네이버 url에 포함시켜 이동 + 검색 
        
    name = res_df['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))

        
    time.sleep(3)  

    ## 화면이 안넘어가고 검색결과에 머물러 있을 때는 entryIframe 이 없음
    ## 그래서 검색어 쳤을때 entryIframe 있으면 --> 화면이 넘어갔다는 뜻이고 
    ## 없으면 아직 안넘어간 뜻이니깐 searchIframe 에서 클릭해서 넘어가야함 .
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
            
            time.sleep(wait_time) 
            
            # 가게 이름, 변수로 지정
            # 메뉴와 가격이 존재하는지의 여부를 고려 
            # 매장마다 메뉴를 보여주는 테마가 2가지 존재하기 때문에 이를 고려

            restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
            category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
            address = driver.find_element(By.CLASS_NAME,'LDgIH').text
            if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                price = driver.find_element(By.CLASS_NAME,'mkBm3').text
            elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                price = driver.find_element(By.CLASS_NAME,'Yrsei').text
            else : 
                menu = '메뉴없음'
                price = "가격없음"
            
            # 리뷰 긁어오기 
            # PXMot 클래스 요소들을 모두 찾음
            # PXMot 는 리뷰의 클래스 이름인데 별점, 방문자리뷰, 블로그리뷰 총 3개가 존재
            pxmot_elements = driver.find_elements(By.CLASS_NAME, 'PXMot')
            
            visitor_review = 0
            blog_review = 0
            # PXMot 가 존재할 때 (별점 방문자 블로그 중  뭐라도 존재)
            if pxmot_elements:
                for element in pxmot_elements:
                    element_text = element.text
                    if element_text.startswith('별점'):
                        # 별점이 있으면 다음 요소 검사
                        continue
                    elif element_text.startswith('방문자리뷰'):
                        # 방문자 리뷰가 존재하는 경우
                        review_text = element_text.replace('방문자리뷰', '')
                        # 쉼표(,) 제거 후 정수로 변환
                        try : 
                            visitor_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            visitor_review = 0
                    elif element_text.startswith('블로그리뷰'):
                        # 블로그 리뷰가 존재하는 경우
                        review_text = element_text.replace('블로그리뷰', '')
                        # 쉼표(,) 제거 후 정수로 변환
                        try :
                            blog_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            blog_review = 0
            # PXMot 가 없을 때(별점 방문자 블로그 중  뭐라도 존재x)
            else  : 
                visitor_review = 0
                blog_review = 0      
        else: 
            # searchIframe 프레임 
            # searchIframe 프레임으로 가야하는 경우는 검색했을 때 결과가 바로 안나오고 식당리스트가 먼저나오는 경우
            # 그래서 searchIFrame에서 첫번쨰 식당을 선택하고 그다음에 entryIFrame으로 들어가야함
            searchIframe = driver.find_element(By.ID,'searchIframe')
            driver.switch_to.frame(searchIframe)


            ## 클릭 
            driver.find_element(By.CLASS_NAME,'YwYLL').click()
            
            # 기본프레임 
            time.sleep(wait_time)
            driver.switch_to.default_content()
        
            # entryIframe 프레임 
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)

            time.sleep(wait_time) 
            # 가게 이름, 변수로 지정
            restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
            category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
            address = driver.find_element(By.CLASS_NAME,'LDgIH').text

            # 메뉴와 가격이 존재하는지의 여부를 고려 
            # 매장마다 메뉴를 보여주는 테마가 2가지 존재하기 때문에 이를 고려

            if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                price = driver.find_element(By.CLASS_NAME,'mkBm3').text
            elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                price = driver.find_element(By.CLASS_NAME,'Yrsei').text
            else : 
                menu = '메뉴없음'
                price = "가격없음"

            # 리뷰 수 긁어오기
            # PXMot 클래스 요소들을 모두 찾음
            # PXMot 는 리뷰의 클래스 이름인데 별점, 방문자리뷰, 블로그리뷰 총 3개가 존재
            pxmot_elements = driver.find_elements(By.CLASS_NAME, 'PXMot')
        
            # PXMot 가 존재할 때 (별점 방문자 블로그 중  뭐라도 존재)
            if pxmot_elements:
                for element in pxmot_elements:
                    element_text = element.text
                    if element_text.startswith('별점'):
                        # 별점이 있으면 다음 요소 검사
                        continue
                    elif element_text.startswith('방문자리뷰'):
                        # 방문자 리뷰가 존재하는 경우
                        review_text = element_text.replace('방문자리뷰', '')
                        # 쉼표(,) 제거 후 정수로 변환
                        try : 
                            visitor_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            visitor_review = 0
                    elif element_text.startswith('블로그리뷰'):
                        # 블로그 리뷰가 존재하는 경우
                        review_text = element_text.replace('블로그리뷰', '').strip()
                        # 쉼표(,) 제거 후 정수로 변환
                        try :
                            blog_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            blog_review = 0
            # PXMot 가 없을 때(별점 방문자 블로그 중  뭐라도 존재x)
            else  : 
                visitor_review = 0
                blog_review = 0   



    ## 예외처리 : vscode 상태가 안좋아서 그냥 검색이 안된경우 이거나 진짜 없는 식당일 때, 문제가 뭔지 확인 
    except Exception as e:
        if 'no such element: Unable to locate element: {"method":"css selector","selector":"[id="searchIframe"]"' in str(e):
            restaurant_name = name 
            category_name = '건물에_매장이_여러개'
            address = '전처리필요'
            menu = '메뉴없음'
            price = "가격없음"
            visitor_review = 0
            blog_review = 0     
        elif 'no such element: Unable to locate element: {"method":"css selector","selector":".YwYLL"}' in str(e):
            restaurant_name = name 
            category_name = '주소에러'
            address = '전처리필요'
            menu = '메뉴없음'
            price = "가격없음"
            visitor_review = 0
            blog_review = 0         
        else: 
            restaurant_name = name 
            category_name = '재확인'
            address = '전처리필요'
            menu = '메뉴없음'
            price = "가격없음"
            visitor_review = 0
            blog_review = 0
        print(f"에러 메시지: {str(e)}")


    ## 리스트에 추가
    restaurant_name_list.append(restaurant_name)
    category_name_list.append(category_name)
    address_list.append(address)
    menu_list.append(menu)
    price_list.append(price)
    visitor_review_list.append(visitor_review)
    blog_review_list.append(blog_review)

    
res_info_df = pd.DataFrame({'매장명':restaurant_name_list ,
                             '업태구분' :category_name_list ,
                             '주소' :address_list ,
                             '메뉴': menu_list,
                             '가격':price_list,
                             '방문자리뷰':visitor_review_list,
                             '블로그리뷰':blog_review_list,
                             })
#driver.quit()

res_info_df

진행 중:   0%|          | 0/2 [00:00<?, ?it/s]

진행 중:  50%|█████     | 1/2 [00:04<00:04,  4.01s/it]

에러 메시지: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="searchIframe"]"}
  (Session info: chrome=123.0.6312.58)



진행 중: 100%|██████████| 2/2 [00:07<00:00,  3.83s/it]

에러 메시지: Message: no such element: Unable to locate element: {"method":"css selector","selector":".YwYLL"}
  (Session info: chrome=123.0.6312.58)



,매장명,업태구분,주소,메뉴,가격,방문자리뷰,블로그리뷰
0,언주로134길 29 폭풍갈비,건물에_매장이_여러개,전처리필요,메뉴없음,가격없음,0,0
1,"도산대로13길 14, 비즈니스아츠 디자인",주소에러,전처리필요,메뉴없음,가격없음,0,0


In [ ]:
## 결과 데이터에 검색어 칼럼 추가
## 추후에 인덱스로 쓸 예정이기 떄문

res_info_df['검색어'] = res_df['검색어']
res_info_df.to_excel(f'./data/restaurant_info_data(2)/restaurant_info_data/restaurant_tag_df_전처리필요.xlsx', index=False)

# 100개 기준 16분 